# Dueling Double DQN

**Dueling Double DQN** combină două idei puternice:
1. **Dueling Network Architecture**  
2. **Double DQN (separarea acțiunii de evaluare)**

Această combinație produce unul dintre cei mai stabili și performanți algoritmi
value-based pentru control discret.

---

## 🎭 1. Dueling Network Architecture

În loc ca rețeaua să prezică direct Q-values pentru fiecare acțiune, Dueling DQN
împarte estimarea în două fluxuri:

### 🔹 Value Stream (V)

Evaluează „valoarea” stării:

$$
V(s)
$$

### 🔹 Advantage Stream (A)

Evaluează avantajul fiecărei acțiuni:

$$
A(s, a)
$$

### 🔹 Combinația finală (normalizare a avantajelor)

$$
Q(s,a) = V(s) + A(s,a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s,a')
$$

Această formulă ajută în stările unde *acțiunea exactă nu contează foarte mult*.

---

## 🎯 2. Double DQN

Double DQN previne supraestimarea Q-values separând alegerea acțiunii de evaluarea ei:

- Rețeaua **online** alege acțiunea  
- Rețeaua **target** o evaluează  

$$
y = r + \gamma \, Q_{\text{target}}
\big(s',\, \arg\max_a Q_{\text{online}}(s',a)\big)
$$

Acest mecanism stabilizează puternic învățarea.

---

## 🔧 3. Soft Target Updates (τ)

În loc de o copie brutală a rețelei online în target, folosim un update lin:

$$
\theta_{\text{target}} \leftarrow 
\tau \, \theta_{\text{online}} +
(1 - \tau) \, \theta_{\text{target}}
$$

unde τ ≈ 0.005.

Avantaje:
- stabilitate mult mai mare  
- convergență lină  
- folosit în algoritmii moderni (SAC, TD3)

---

## 🚀 Beneficii majore

✔ Învățare mai rapidă în stări „ne-informative”  
✔ Target fără supraestimări  
✔ Variabilitate redusă  
✔ Performanță excelentă pe probleme complexe precum:
- LunarLander  
- MountainCar  
- Acrobot  
- Atari  

---

## 🧩 Pipeline (pe scurt)

1. Selectăm acțiunea cu **rețeaua online**  
2. Calculăm valoarea țintă cu **rețeaua target**  
3. Combinăm V + A → Q prin formula dueling  
4. Facem gradient descent  
5. Aplicăm soft update (τ)  

---

## 📌 Pe scurt

**Dueling Double DQN =  
Double DQN (stabil) + Dueling Network (eficient) + Soft Updates (modern)**

Una dintre cele mai bune arhitecturi value-based pentru control discret și
componentă esențială în **Rainbow DQN**.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers

# ============================================================
# Replay Buffer
# ============================================================

class Memory(object):
    def __init__(self, memory_size: int) -> None:
        self.memory_size = memory_size
        self.buffer = deque(maxlen=memory_size)

    def add(self, experience) -> None:
        self.buffer.append(experience)

    def size(self):
        return len(self.buffer)

    def sample(self, batch_size: int):
        batch_size = min(batch_size, len(self.buffer))
        indexes = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[i] for i in indexes]


# ============================================================
# Dueling Network
# ============================================================

class Qnetwork(Model):
    def __init__(self):
        super(Qnetwork, self).__init__()

        self.fc1 = layers.Dense(64, activation='relu')
        self.fc_value = layers.Dense(256, activation='relu')
        self.fc_adv = layers.Dense(256, activation='relu')

        self.value = layers.Dense(1)     # V(s)
        self.adv = layers.Dense(2)       # A(s,a) — CartPole: 2 actions

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = self.fc1(x)

        value = self.fc_value(x)
        adv = self.fc_adv(x)

        value = self.value(value)
        adv = self.adv(adv)

        adv_mean = tf.reduce_mean(adv, axis=1, keepdims=True)
        Q = value + adv - adv_mean  # dueling combine

        return Q

    def select_action(self, obs):
        obs = np.array(obs, dtype=np.float32)[np.newaxis, :]
        Q = self.call(obs)
        return int(tf.argmax(Q[0]).numpy())


# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
INITIAL_EPSILON = 0.99
FINAL_EPSILON = 0.0001
EXPLORATION_STEPS = 20000
REPLAY_MEMORY = 50000
BATCH = 32
UPDATE_STEPS = 10
LR = 1e-3 * 5
NUM_EPISODES = 1000

# ============================================================
# Main
# ============================================================

if __name__ == "__main__":
    tf.keras.backend.set_floatx('float32')

    memory_replay = Memory(REPLAY_MEMORY)
    epsilon = INITIAL_EPSILON
    learn_steps = 0
    begin_learn = False

    writer = SummaryWriter('ddqn-tf2')

    env = gym.make("CartPole-v1")

    online_q = Qnetwork()
    target_q = Qnetwork()
    target_q.set_weights(online_q.get_weights())

    mse = tf.keras.losses.MeanSquaredError()
    optim = optimizers.Adam(LR)

    reward_history = []

    for epoch in range(NUM_EPISODES):

        state, info = env.reset()
        episode_reward = 0

        for t in range(200):

            # ------------------------------------------------
            # Epsilon-greedy
            # ------------------------------------------------
            if random.random() < epsilon:
                action = random.randint(0, 1)
            else:
                action = online_q.select_action(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward

            # store experience
            memory_replay.add((state, next_state, action, reward, done))
            state = next_state

            # ------------------------------------------------
            # Learn
            # ------------------------------------------------
            if memory_replay.size() > 100:
                if not begin_learn:
                    print("Learning starts!")
                    begin_learn = True

                learn_steps += 1

                # sync target network
                if learn_steps % UPDATE_STEPS == 0:
                    target_q.set_weights(online_q.get_weights())

                batch = memory_replay.sample(BATCH)
                batch_state, batch_next_state, batch_action, batch_reward, batch_done = zip(*batch)

                batch_state = np.asarray(batch_state, dtype=np.float32)
                batch_next_state = np.asarray(batch_next_state, dtype=np.float32)
                batch_action = np.asarray(batch_action, dtype=np.int32)
                batch_reward = np.asarray(batch_reward, dtype=np.float32)
                batch_done = np.asarray(batch_done, dtype=np.float32)

                # -------------------------------
                # Double DQN update
                # -------------------------------
                next_action_online = tf.argmax(online_q(batch_next_state), axis=1)
                next_action_online = tf.cast(next_action_online, tf.int32)
                next_index = tf.stack([tf.range(tf.shape(next_action_online)[0]), next_action_online], axis=1)

                q_target_next = target_q(batch_next_state)
                q_target_selected = tf.gather_nd(q_target_next, next_index)

                y = batch_reward + (1.0 - batch_done) * GAMMA * q_target_selected

                # -------------------------------
                # Compute loss
                # -------------------------------
                with tf.GradientTape() as tape:
                    q_pred = online_q(batch_state)
                    batch_action_idx = tf.stack([tf.range(tf.shape(batch_action)[0]), batch_action], axis=1)
                    q_pred_selected = tf.gather_nd(q_pred, batch_action_idx)
                    loss = mse(q_pred_selected, y)

                grads = tape.gradient(loss, online_q.trainable_variables)
                optim.apply_gradients(zip(grads, online_q.trainable_variables))

                # update epsilon
                if epsilon > FINAL_EPSILON:
                    epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORATION_STEPS

                writer.add_scalar("loss", float(loss), learn_steps)

            if done:
                break

        reward_history.append(episode_reward)
        writer.add_scalar("episode_reward", episode_reward, epoch)

        # ----------------------------------------------------
        # LIVE PLOT
        # ----------------------------------------------------
        # ----------------------------
        # LIVE PLOT
        # ----------------------------
        if epoch % 5 == 0:
            clear_output(wait=True)
            plt.figure(figsize=(10,4))

            # Reward
            plt.plot(
                reward_history,
                label="Reward",
                color="blue",
                alpha=0.3,  
                linewidth=1
            )

            # Moving average
            if len(reward_history) > 20:
                ma = np.convolve(reward_history, np.ones(20)/20, mode='valid')
                plt.plot(
                    range(19, len(reward_history)),
                    ma,
                    label="Moving Avg (20 eps)",
                    color="orange",
                    linewidth=2.5
                )

            plt.title("Dueling Double DQN — Training Progress")
            plt.xlabel("Episode")
            plt.ylabel("Reward")
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()


# Optimizări pentru Dueling Double DQN

## ✔ Soft Updates (τ = 0.005)

Înlocuiește copia bruscă a rețelei target cu o actualizare lină:

$$
\theta_{\text{target}} \leftarrow 
\tau\,\theta_{\text{online}} + (1-\tau)\,\theta_{\text{target}}
$$

→ reduce oscilațiile și stabilizează învățarea.

---

## ✔ Gradient Clipping

Limitează norma gradientului pentru a evita divergența și valori NaN.  
Asigură update-uri mici, controlate și stabile.

---

## ✔ Q-value Clipping

Aplicăm limitare pe predicțiile Q:

$$
Q \leftarrow \text{clip}(Q,\,-500,\,500)
$$

→ previne „explozia” valorilor în stări instabile.

---

## ✔ Learning Rate mic + Rețea mai adâncă

- **LR = 3e-4**  
- două straturi dense a câte **128 neuroni**

→ stabilitate mai mare și reprezentare mai bună a funcției Q.

---

## ✔ Smooth Epsilon Decay (0.995)

Agentul reduce epsilon treptat:

- explorare → exploatare fără salturi bruște  
- comportament mai natural și mai stabil

---

## ✔ Replay Buffer Vectorizat

Memoria folosește matrici NumPy, nu liste:

- sampling rapid  
- update-uri eficiente  
- integrare optimă cu TensorFlow  

---

## 📌 Pe scurt

Aceste optimizări fac Dueling Double DQN **mult mai stabil, mai rapid și mai robust**, oferind un training consistent chiar și în environmente dificile.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers


# ============================================================
# Replay Buffer (optimized)
# ============================================================

class ReplayBuffer:
    def __init__(self, size, state_dim):
        self.mem_size = size
        self.mem_cntr = 0

        self.state_memory = np.zeros((size, state_dim), dtype=np.float32)
        self.next_state_memory = np.zeros((size, state_dim), dtype=np.float32)
        self.action_memory = np.zeros(size, dtype=np.int32)
        self.reward_memory = np.zeros(size, dtype=np.float32)
        self.done_memory = np.zeros(size, dtype=np.float32)

    def store(self, state, next_state, action, reward, done):
        idx = self.mem_cntr % self.mem_size
        self.state_memory[idx] = state
        self.next_state_memory[idx] = next_state
        self.action_memory[idx] = action
        self.reward_memory[idx] = reward
        self.done_memory[idx] = done
        self.mem_cntr += 1

    def sample(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        return (
            self.state_memory[batch],
            self.next_state_memory[batch],
            self.action_memory[batch],
            self.reward_memory[batch],
            self.done_memory[batch]
        )


# ============================================================
# Dueling Network (optimized)
# ============================================================

class DuelingQNetwork(Model):
    def __init__(self, n_actions):
        super().__init__()

        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(128, activation='relu')

        self.value_fc = layers.Dense(128, activation='relu')
        self.adv_fc = layers.Dense(128, activation='relu')

        self.value = layers.Dense(1)
        self.advantage = layers.Dense(n_actions)

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = self.fc1(x)
        x = self.fc2(x)

        V = self.value(self.value_fc(x))
        A = self.advantage(self.adv_fc(x))

        A_mean = tf.reduce_mean(A, axis=1, keepdims=True)
        Q = V + (A - A_mean)

        return tf.clip_by_value(Q, -500, 500)

    def act(self, obs):
        obs = obs[np.newaxis, :].astype(np.float32)
        Q = self.call(obs)[0]
        return int(tf.argmax(Q).numpy())


# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
LR = 3e-4
BATCH = 32
MEM_SIZE = 50000
NUM_EPISODES = 600

INITIAL_EPS = 1.0
FINAL_EPS = 0.02
EPS_DECAY = 0.995

TAU = 0.005   # soft update


# ============================================================
# Soft update function
# ============================================================

def soft_update(target, online, tau=TAU):
    target_weights = target.get_weights()
    online_weights = online.get_weights()
    new_weights = []

    for tw, ow in zip(target_weights, online_weights):
        new_weights.append(tw * (1 - tau) + ow * tau)

    target.set_weights(new_weights)


# ============================================================
# Training Loop
# ============================================================

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

buffer = ReplayBuffer(MEM_SIZE, state_dim)
online = DuelingQNetwork(n_actions)
target = DuelingQNetwork(n_actions)
target.set_weights(online.get_weights())

optimizer = optimizers.Adam(LR)
mse = tf.keras.losses.MeanSquaredError()

epsilon = INITIAL_EPS
reward_history = []

for episode in range(NUM_EPISODES):

    state, _ = env.reset()
    ep_reward = 0

    for step in range(200):

        # ----------------------------
        # Epsilon-greedy action
        # ----------------------------
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = online.act(state)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        ep_reward += reward

        # store transition
        buffer.store(state, next_state, action, reward, float(done))
        state = next_state

        # ----------------------------
        # TRAINING
        # ----------------------------
        if buffer.mem_cntr > 1000:

            states, next_states, actions, rewards, dones = buffer.sample(BATCH)

            # Double DQN — FIXED with correct dtypes
            next_actions = tf.argmax(
                online(next_states), axis=1, output_type=tf.int32
            )

            indices = tf.stack(
                [tf.range(BATCH, dtype=tf.int32), next_actions], axis=1
            )

            next_Q = tf.gather_nd(target(next_states), indices)
            y = rewards + GAMMA * (1 - dones) * next_Q

            # ----------------------------
            # Compute loss
            # ----------------------------
            with tf.GradientTape() as tape:
                q_pred = online(states)
                action_idx = tf.stack(
                    [tf.range(BATCH, dtype=tf.int32), actions], axis=1
                )
                pred = tf.gather_nd(q_pred, action_idx)
                loss = mse(pred, y)

            grads = tape.gradient(loss, online.trainable_variables)
            grads = [tf.clip_by_norm(g, 5.0) for g in grads]
            optimizer.apply_gradients(zip(grads, online.trainable_variables))

            # Soft update for stability
            soft_update(target, online)

        if done:
            break

    # ----------------------------
    # Epsilon decay
    # ----------------------------
    epsilon = max(FINAL_EPS, epsilon * EPS_DECAY)
    reward_history.append(ep_reward)

    # ----------------------------
    # LIVE PLOT
    # ----------------------------
    if episode % 5 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(10,4))

        # Reward
        plt.plot(
            reward_history,
            label="Reward",
            color="blue",
            alpha=0.3,  
            linewidth=1
        )

        # Moving average
        if len(reward_history) > 20:
            ma = np.convolve(reward_history, np.ones(20)/20, mode='valid')
            plt.plot(
                range(19, len(reward_history)),
                ma,
                label="Moving Avg (20 eps)",
                color="orange",
                linewidth=2.5
            )

        plt.title("Dueling Double DQN — Training Progress")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
